###### References: 
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python, 2nd Edition, by Luciano Ramalho. Chapter 24: Class Metaprogramming

# Classes as Objects
## type: The Built-In Class Factory

When Python reads a class statement, it calls type to  buildd the class object with 3 parameters: *name*, *bases*, *dict*.

    < type( name, bases, dict ) >

In [1]:
type(7)

int

In [2]:
type(int)

type

In [3]:
type(OSError)

type

In [4]:
class Whatever:
    pass

In [5]:
type(Whatever)

type

## A  Class Factory Function

### record Factory

In [6]:
from typing import Union, Any
from collections.abc import Iterable, Iterator

FieldNames = Union[str, Iterable[str]]  # User can provide field names as single or an iterable of strings

def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:  # accept arguments and returns a type

    slots = parse_identifiers(field_names)  # build a tuple of attribute names

    def __init__(self, *args, **kwargs) -> None:  # this function will be the __init__ method in the new class
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:  # Yield the field values in the order given by __slots__
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):  # produce the nice repr, iiterating over __slots__ and self
        values = ', '.join(f'{name}={value!r}'
            for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'

    cls_attrs = dict(  # assemble a dictionary of class attributes
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (object,), cls_attrs)  # build and return the new class, calling type constructor


def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()  # Convert names separated by spaces or commas
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')
    return tuple(names)

### Testing record factory:

In [7]:
Dog = record_factory('Dog', 'name weight owner')

In [8]:
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [9]:
name, weight, _ = rex
name, weight

('Rex', 30)

In [10]:
"{2}'s dog weighs {1}kg".format(*rex)

"Bob's dog weighs 30kg"

In [11]:
rex.weight = 32
rex

Dog(name='Rex', weight=32, owner='Bob')

In [12]:
Dog.__mro__

(__main__.Dog, object)

### Introducing `__init_subclass__`

`__init_subclass__` andd `__set_name__` read type hints in user's `class` statement; and allow static type checkers to validate code via type hints. (Whereas `typing.NamedTuple` and `@dataclass` do not)

In [13]:
from collections.abc import Callable  # typing.Callable depreciated since 3.9
from typing import Any, NoReturn, get_type_hints

In [14]:
class Field:
    def __init__(self, name: str, constructor: Callable) -> None:  # minimal type hint for Callable
        if not callable(constructor) or constructor is type(None):  # for runtime checking
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:  # Ellipsis built-in object
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)  # otherwise constructor with given value
            except (TypeError, ValueError) as e:  # raise error
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value  # value stored in the instance.__dict__

In [15]:
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:  # to support Python<3.10
        return get_type_hints(cls)

    def __init_subclass__(subclass) -> None:  # __init_subclass__ is called when a subclass of the current class is defined
        super().__init_subclass__()           
        for name, constructor in subclass._fields().items():   # iterate over field name andd constructor
            setattr(subclass, name, Field(name, constructor))  # creating an attribute on subclass

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():             # for each name in the class fields
            value = kwargs.pop(name, ...)       # get the corresponding kwargs and remove
            setattr(self, name, value)          # the setattr call triggers Checked.__setattr__
        if kwargs:                              # if remaining items do not match
            self.__flag_unknown_attrs(*kwargs)  # fail and raise error

    def __setattr__(self, name: str, value: Any) -> None:  # intercept all attemps to set an instance attribute
        if name in self._fields():              # if attriibute name known fetch the corresponding descriptor
            cls = self.__class__
            descriptor = getattr(cls, name)
            descriptor.__set__(self, value)     # call set explicitly since we intercepted
        else:                                   # unknown attribute
            self.__flag_unknown_attrs(name)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:  # exception to noReturn, build error message
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:  # Create a dict from the attributes
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:  # implementing a nice repr
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

#### Build a Movie class:

In [16]:
class Movie(Checked):
    title: str
    year: int
    box_office: float

In [17]:
movie = Movie(title='The Godfather', year=1972, box_office=137)
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

## Enhancing Classes with a Class Decorator

In [18]:
def checked(cls: type) -> type:  # hint: takes a class, returns a class
    for name, constructor in _fields(cls).items():    # top-level function defined later
        setattr(cls, name, Field(name, constructor))  # replacing each attribute returned by _fields with a Field descriptor instance

    cls._fields = classmethod(_fields)  # type: ignore  # Build a class method from _fields and addd it to  the decorated class

    instance_methods = (  # module level functions that  will become instance methods of the decorated class
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    for method in instance_methods:  # Add each instance_methods  to cls
        setattr(cls, method.__name__, method)

    return cls  # return the decorated cls

def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self: Any, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self: Any, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self: Any, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} has no attribute{plural} {extra}')

def _asdict(self: Any) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self: Any) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'

In [19]:
@checked
class Movie:
    title: str
    year: int
    box_office: float

In [20]:
movie = Movie(title='The Godfather', year=1972, box_office=137)

In [21]:
movie.title

'The Godfather'

In [22]:
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

## Evaluation Time Experiments

In [23]:
import builderlib

@ builderlib module start
@ Builder body
@ Descriptor body
@ builderlib module end


In [24]:
import evaldemo

# evaldemo module start
# Klass body
@ Descriptor.__init__(<Descriptor instance>)
@ Descriptor.__set_name__(<Descriptor instance>, <class 'evaldemo.Klass'>, 'attr')
@ Builder.__init_subclass__(<class 'evaldemo.Klass'>)
@ deco(<class 'evaldemo.Klass'>)
# evaldemo module end


# Metaclasses 101
A metaclass is a class factory.  
A metaclasss is written as a class, i.e. a metaclass is a class whose instances are classes.

In [25]:
str.__class__

type

In [26]:
type.__class__

type

<img src="Meta class instance.png" width="75%">

In [27]:
from collections.abc import Iterable

In [28]:
Iterable.__class__

abc.ABCMeta

In [29]:
import abc
from abc import ABCMeta

In [30]:
ABCMeta.__class__

type

## A Metaclass Hack with `__prepare__`
Autogenerates numeric constants:

In [31]:
class WilyDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__next_value = 0

    def __missing__(self, key):
        if key.startswith('__') and key.endswith('__'):
            raise KeyError(key)
        self[key] = value = self.__next_value
        self.__next_value += 1
        return value

In [32]:
wd = WilyDict()

In [33]:
len(wd)

0

In [34]:
wd['first']

0

In [35]:
wd

{'first': 0}

In [36]:
wd['second']

1

In [37]:
wd['third']

2

In [38]:
len(wd)

3

In [39]:
wd

{'first': 0, 'second': 1, 'third': 2}

In [40]:
wd["__magic__"]

KeyError: '__magic__'

In [41]:
class AutoConstMeta(type):
    def __prepare__(name, bases, **kwargs):
        return WilyDict()

class AutoConst(metaclass=AutoConstMeta):
    pass

In [42]:
class Flavor(AutoConst):
    banana
    coconut
    vanilla

In [43]:
Flavor.vanilla

2

In [44]:
(Flavor.banana, Flavor.coconut)

(0, 1)